# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p sc:name ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-22a4ae1fc8-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("F1 pilots explorative search") 

Consider the following exploratory information need:

> explore F1 pilots, their nationality, racing teams, and the country of the parent organization of the team

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P17`     | country       | predicate |
| `wd:Q10497835`| F1 Team       | node |
| `wd:Q10841764`| F1 Driver       | node |
| `wd:Q9671`    | Michael Schumacher      | node |
| `wd:Q169898`  | Scuderia Ferrari         | node |
| `wd:Q27586`   |  Ferrari         | node |
| `wd:Q183`     | Germany       | node |


Also consider

```
?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .
```

is the BGP to retrieve all **citizens of Germany**

```
?p wdt:P17 wd:Q183 .
?p wdt:P31 wd:Q10497835 .

``` 

is the BGP to retrieve all **F1 teams of Germany**

## Workload Goals

1. Identify the BGP for obtaining the team of a given  F1 driver  and also the F1 competitions in which they participated

2. Identify the BGP that connects a F1 team to a parent company

3. How many F1 drivers and F1 teams exist in each country?

4. Compare number of F1 drivers to their nationality
 
   4.1 Is there some driver that joined a team in their own country?
   
   4.2 Which team employed the larger number of drivers?
   
   4.3 For each country , which are the teams and drivers with the highest number of F1 race participations


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P106 wd:Q10841764 .
?p wdt:P27 wd:Q183 .

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '58')]


1

## Task 1

Look for some properties of the F1 driver and teams

In [4]:
#relations for the drivers
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
wd:Q10841764 ?p ?t .
?p sc:name ?name .
FILTER(!isLiteral(?o))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('name', 'field of this occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]


7

In [5]:
#relations for the teams
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
wd:Q10497835 ?p ?t .
?p sc:name ?name .
FILTER(!isLiteral(?o))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]


5

From this find the sport `Formula One`

In [6]:
#relations for the teams
queryString = """
SELECT DISTINCT ?t ?name
WHERE { 
wd:Q10497835 wdt:P641 ?t .
?t sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q1968'), ('name', 'Formula One')]


1

`wd:Q1968` is the node Formula One

Find the outgoing relations from an F1 Driver that point to an instance of F1 Team

In [7]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 

?t wdt:P31 wd:Q10497835 .

?a wdt:P106 wd:Q10841764.
?a ?p ?t .
?p sc:name ?name .
FILTER(!isLiteral(?o))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]


3

Looking in deeper about relation "member of" and "member of sports team" but seems that "member of" is the F1 team.

In [8]:
queryString = """
SELECT DISTINCT ?a ?driver ?team
WHERE { 

?t wdt:P31 wd:Q10497835 .
?a wdt:P106 wd:Q10841764;
    wdt:P54 ?t .
?a sc:name ?driver .
?t sc:name ?team .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q172057'), ('driver', 'Reg Parnell'), ('team', 'Alfa Romeo')]
[('a', 'http://www.wikidata.org/entity/Q544856'), ('driver', 'Carlos Sainz Jr'), ('team', 'Scuderia Toro Rosso')]
[('a', 'http://www.wikidata.org/entity/Q544856'), ('driver', 'Carlos Sainz Jr'), ('team', 'McLaren')]
[('a', 'http://www.wikidata.org/entity/Q544856'), ('driver', 'Carlos Sainz Jr'), ('team', 'Renault F1 Team')]
[('a', 'http://www.wikidata.org/entity/Q3128'), ('driver', 'Graham Hill'), ('team', 'British Racing Motors')]
[('a', 'http://www.wikidata.org/entity/Q3128'), ('driver', 'Graham Hill'), ('team', 'Equipe Matra Sports')]
[('a', 'http://www.wikidata.org/entity/Q3128'), ('driver', 'Graham Hill'), ('team', 'Brabham')]
[('a', 'http://www.wikidata.org/entity/Q3153'), ('driver', 'John Surtees'), ('team', 'British Racing Motors')]
[('a', 'http://www.wikidata.org/entity/Q3153'), ('driver', 'John Surtees'), ('team', 'Honda')]
[('a', 'http://www.wikidata.org/entity/Q3153'

100

In [9]:

queryString = """
SELECT DISTINCT ?a ?pilot ?member
WHERE { 

?t wdt:P31 wd:Q10497835 .
?a wdt:P106 wd:Q10841764;
    wdt:P463 ?t .
?a sc:name ?pilot .
?t sc:name ?member .
} 
LIMIT 1000
"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q171432'), ('pilot', 'Didier Pironi'), ('member', 'Tyrrell Racing')]
[('a', 'http://www.wikidata.org/entity/Q171432'), ('pilot', 'Didier Pironi'), ('member', 'Scuderia Ferrari')]
[('a', 'http://www.wikidata.org/entity/Q171526'), ('pilot', 'Jean-Pierre Beltoise'), ('member', 'British Racing Motors')]


3

The relation "member of sports team" (`P54`) is the one that tell me the  F1 team of a given F1 driver

Now I have to find the F1 competitions in which a F1 driver compete looking for the relations of a driver.

In [10]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 

?a wdt:P106 wd:Q10841764;
    ?pr ?t.
#?t wdt:P31 ?ist.

#?ist sc:name ?ista .
?pr sc:name ?prop .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P3927'), ('prop', 'eWRC-results.com racer ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P9129'), ('prop', 'Data Collection of the Hungarian Peerage ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P6682'), ('prop', 'Motorsports Hall of Fame of America ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P8161'), ('prop', 'CONI honoured ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1006'), ('prop', 'Nationale Thesaurus voor Auteurs ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1015'), ('prop', 'NORAF ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P102'), ('prop', 'member of political party')]
[('pr', 'http://www.wikidata.org/prop/direct/P103'), ('prop', 'native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('prop', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('prop', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P108'), ('prop', 'employer')]
[('pr', 'http://w

100

Something interest: `P1344`, participant in. Looking for what is on the other side of the relation.

In [11]:
queryString = """
SELECT DISTINCT ?t ?prop ?ist ?ista
WHERE { 

?a wdt:P106 wd:Q10841764;
    wdt:P1344 ?t .
?t wdt:P31 ?ist.
?ist sc:name ?ista .
?t sc:name ?prop .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q149844'), ('prop', 'GP3 Series'), ('ist', 'http://www.wikidata.org/entity/Q60147807'), ('ista', 'automobile racing series')]
[('t', 'http://www.wikidata.org/entity/Q17455748'), ('prop', '2016 Formula One World Championship'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q22833485'), ('prop', "2015 FIA World Endurance Drivers' Championship"), ('ist', 'http://www.wikidata.org/entity/Q22833327'), ('ista', 'international championship award')]
[('t', 'http://www.wikidata.org/entity/Q69343'), ('prop', '1974 Formula One season'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q69641'), ('prop', '1989 Formula One season'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q8415'), ('prop', '1960 Summer Olympics'), ('ist', 'http://www.w

98

The relation give me all the season in which a driver partecipate. But there is also other sports that are not about F1, so I will apply a REGEX filter.

In [12]:
queryString = """
SELECT DISTINCT ?t ?prop ?ist ?ista
WHERE { 

?a wdt:P106 wd:Q10841764;
    wdt:P1344 ?t .
?t wdt:P31 ?ist.

?ist sc:name ?ista .

?t sc:name ?prop .
FILTER(REGEX(?prop,\".*[F,f]ormula [O,o]ne.*\"))
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q17455748'), ('prop', '2016 Formula One World Championship'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q69343'), ('prop', '1974 Formula One season'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q69641'), ('prop', '1989 Formula One season'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q69229'), ('prop', '1967 Formula One season'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q69633'), ('prop', '1988 Formula One season'), ('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]
[('t', 'http://www.wikidata.org/entity/Q69913'), ('prop', '2008 Formula One World Championship'), ('ist', 'http://www.wikidata.org/entity/Q27020041

34

In [13]:
queryString = """
SELECT DISTINCT ?t ?prop
WHERE { 

?a wdt:P106 wd:Q10841764;
    wdt:P1344 ?t .
?t wdt:P31 ?ist.
?t sc:name ?prop .
FILTER(REGEX(?prop,\".*[F,f]ormula [O,o]ne.*\"))
} 
ORDER BY desc (?prop)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('t', 'http://www.wikidata.org/entity/Q1968'), ('prop', 'Formula One')]
[('t', 'http://www.wikidata.org/entity/Q41173535'), ('prop', '2019 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q23051084'), ('prop', '2018 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q20755615'), ('prop', '2017 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q17455748'), ('prop', '2016 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q14293115'), ('prop', '2015 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q69969'), ('prop', '2014 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q69356'), ('prop', '2013 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q69959'), ('prop', '2012 Formula One World Championship')]
[('t', 'http://www.wikidata.org/entity/Q69943'), ('prop', '2011 Formula One World Championship')]
[('t', 'http://www.wik

33

Maybe Q1968 (Formula One) is something acting as a superclass

In [14]:
queryString = """
SELECT DISTINCT ?pr ?pro
WHERE { 

?a wdt:P106 wd:Q10841764;
    wdt:P1344 ?t .
?t ?pr wd:Q1968.
?t sc:name ?prop .
?pr sc:name ?pro .
FILTER(REGEX(?prop,\".*[F,f]ormula [O,o]ne.*\"))
} 
ORDER BY desc (?prop)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P641'), ('pro', 'sport')]
[('pr', 'http://www.wikidata.org/prop/direct/P3450'), ('pro', 'sports season of league or competition')]


2

No is the sport. So starting for the Q1968 that is the sport looking for some properties

In [15]:
queryString = """
SELECT DISTINCT ?pr ?pro
WHERE { 

?x ?pr wd:Q1968.

?pr sc:name ?pro .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1001'), ('pro', 'applies to jurisdiction')]
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('pro', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P118'), ('pro', 'league')]
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('pro', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('pro', 'participant in')]
[('pr', 'http://www.wikidata.org/prop/direct/P136'), ('pro', 'genre')]
[('pr', 'http://www.wikidata.org/prop/direct/P1535'), ('pro', 'used by')]
[('pr', 'http://www.wikidata.org/prop/direct/P179'), ('pro', 'part of the series')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('pro', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P2094'), ('pro', 'competition class')]
[('pr', 'http://www.wikidata.org/prop/direct/P2416'), ('pro', 'sports discipline competed in')]
[('pr', 'http://www.wikidata.org/prop/direct/P279'), ('pro', 'subclass of')]
[('pr', 'http://www.wikidata

24

P3450 is the predicate that tell me that something is a sports season

In [16]:
queryString = """
SELECT DISTINCT ?pr ?pro
WHERE { 

?pr wdt:P3450 wd:Q1968.

?pr sc:name ?pro .
} 
ORDER BY DESC (?pro)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/entity/Q108380238'), ('pro', '2023 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q92821370'), ('pro', '2022 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q72635834'), ('pro', '2021 FIA Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q41174436'), ('pro', '2020 FIA Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q41173535'), ('pro', '2019 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q23051084'), ('pro', '2018 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q20755615'), ('pro', '2017 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q17455748'), ('pro', '2016 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q14293115'), ('pro', '2015 Formula One World Championship')]
[('pr', 'http://www.wikidata.org/entity/Q69969'), ('pro', '2014 Formula On

74

Find the class of the sports season

In [17]:
queryString = """
SELECT DISTINCT ?ist ?ista
WHERE { 

?pr wdt:P3450 wd:Q1968;
    wdt:P31 ?ist.

?ist sc:name ?ista .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q27020041'), ('ista', 'sports season')]


1

So at this point I have to summarize:
- `wd:Q1968` is a formula one sport
- `wdt:P3450` is the predicate named "sports season of league or competition" that allowed me to find a correlation between
  something and a formula one sport season
- `?pr wdt:P3450 wd:Q1968`; gives to me all the sports season of the formula one.

Looking for relations of the sports season outgoing

In [18]:
queryString = """
SELECT DISTINCT ?p ?prop
WHERE { 

?pr wdt:P3450 wd:Q1968;
    ?p ?o.

?p sc:name ?prop .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2321'), ('prop', 'general classification of race participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('prop', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prop', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('prop', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('prop', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('prop', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('prop', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('prop', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('prop', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('prop', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('prop', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), (

35

Looking for relations of the sports season ingoing

In [19]:
queryString = """
SELECT DISTINCT ?p ?prop
WHERE { 

?pr wdt:P3450 wd:Q1968.
 ?o   ?p ?pr.

?p sc:name ?prop .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('prop', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('prop', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('prop', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('prop', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('prop', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('prop', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('prop', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('prop', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('prop', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('prop', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('prop', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('prop', 'league level above')]
[('p', 'http://www.wikidata.org/p

13

The key of all seems to be `P1344`. Looking for the istance of the other side of this reltion. (?o) in the previous query.

I already know that a driver is connected to the season with this relation, but a F1 team?

In [20]:
queryString = """
SELECT DISTINCT ?season ?team
WHERE { 


?t wdt:P31 wd:Q10497835 .
?s wdt:P3450 wd:Q1968.
?t wdt:P1344 ?s.
?t sc:name ?team .
?s sc:name ?season .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

How a team is connected to a season?

In [21]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 


?t wdt:P31 wd:Q10497835 .
?s wdt:P3450 wd:Q1968.
?t ?pr ?s.
?t sc:name ?team .
?pr sc:name ?prop .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Looking at the opposite relation way

In [22]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 


?t wdt:P31 wd:Q10497835 .
?s wdt:P3450 wd:Q1968.
?s ?pr ?t.
?t sc:name ?team .
?pr sc:name ?prop .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('prop', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1923'), ('prop', 'participating team')]
[('pr', 'http://www.wikidata.org/prop/direct/P2318'), ('prop', 'debut participant')]


3

Found `P1923` that tell me the partecipation of a team in a given season.

In [23]:
queryString = """
SELECT DISTINCT ?team ?season
WHERE { 


?t wdt:P31 wd:Q10497835 .
?s wdt:P3450 wd:Q1968;
    wdt:P1923 ?t.
?t sc:name ?team .
?s sc:name ?season .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('team', 'Scuderia Toro Rosso'), ('season', '2017 Formula One World Championship')]
[('team', 'Scuderia Toro Rosso'), ('season', '2016 Formula One World Championship')]
[('team', 'Scuderia Toro Rosso'), ('season', '2018 Formula One World Championship')]
[('team', 'Red Bull Racing'), ('season', '2017 Formula One World Championship')]
[('team', 'Red Bull Racing'), ('season', '2018 Formula One World Championship')]
[('team', 'Red Bull Racing'), ('season', '2016 Formula One World Championship')]
[('team', 'Haas F1 Team'), ('season', '2017 Formula One World Championship')]
[('team', 'Haas F1 Team'), ('season', '2016 Formula One World Championship')]
[('team', 'Haas F1 Team'), ('season', '2018 Formula One World Championship')]
[('team', 'Scuderia Ferrari'), ('season', '2017 Formula One World Championship')]
[('team', 'Scuderia Ferrari'), ('season', '2016 Formula One World Championship')]
[('team', 'Scuderia Ferrari'), ('season', '2018 Formula One World Championship')]
[('team', 'Wil

30

In [24]:
queryString = """
SELECT DISTINCT ?s ?xx
WHERE { 


?t wdt:P31 wd:Q10497835 .
?s wdt:P3450 wd:Q1968.
?t ?pr ?o.
?o ?x ?s.
?o sc:name ?obj .
?s sc:name ?xx .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q68926'), ('xx', '1950 Formula One season')]


1

Try to do the final query!

In [3]:
queryString = """
SELECT DISTINCT ?driver ?team ?season
WHERE { 

#team part
?t wdt:P31 wd:Q10497835 .

#driver part
?a wdt:P106 wd:Q10841764;
    wdt:P54 ?t;
    wdt:P1344 ?s .
?s wdt:P1923 ?t. 
?s sc:name ?season .
?t sc:name ?team .
?a sc:name ?driver .

FILTER(REGEX(?season,\".*[F,f]ormula [O,o]ne.*\"))
}
ORDER by ?a
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('driver', 'Fernando Alonso'), ('team', 'Scuderia Ferrari'), ('season', '2017 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'Renault F1 Team'), ('season', '2018 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'Scuderia Ferrari'), ('season', '2016 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'McLaren'), ('season', '2017 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'McLaren'), ('season', '2016 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'McLaren'), ('season', '2018 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'Renault F1 Team'), ('season', '2016 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'Scuderia Ferrari'), ('season', '2018 Formula One World Championship')]
[('driver', 'Fernando Alonso'), ('team', 'Renault F1 Team'), ('season', '2017 Formula One World Championship')]


9

## Task 2
Identify the BGP that connects a F1 team to a parent company starting from Ferrari

In [26]:
queryString = """
SELECT DISTINCT ?p ?prop
WHERE { 

wd:Q169898 ?p wd:Q27586 .
?p sc:name ?prop .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P749'), ('prop', 'parent organization')]


1

Looking for some relations

In [27]:
queryString = """
SELECT DISTINCT ?p ?prop count(*) AS ?count
WHERE { 

?t wdt:P31 wd:Q10497835;
     ?p ?o .
?p sc:name ?prop .
}
GROUP BY ?p ?prop
ORDER BY DESC (?count)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('prop', 'instance of'), ('count', '136')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('prop', 'country'), ('count', '60')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('prop', 'Freebase ID'), ('count', '55')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('prop', 'Commons category'), ('count', '52')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('prop', "topic's main category"), ('count', '45')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('prop', 'image'), ('count', '42')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('prop', 'inception'), ('count', '34')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('prop', 'Commons gallery'), ('count', '34')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('prop', 'official website'), ('count', '28')]
[('p', 'http://www.wikidata.org/prop/direct/P8687'), ('prop', 'social media followers'), ('count', '26')]
[('p', 'http://www.wikidata.or

73

Found intersting two relation, owned by (`P127`), parent organization (`P749`) and (`P355`). Query the system with both

In [28]:
queryString = """
SELECT DISTINCT ?team ?owner
WHERE { 

?t wdt:P31 wd:Q10497835;
     wdt:P127 ?o .
?t sc:name ?team .
?o sc:name ?owner .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('team', 'Red Bull Racing'), ('owner', 'Red Bull GmbH')]
[('team', 'Alpine F1 Team'), ('owner', 'Renault')]
[('team', 'Williams Grand Prix Engineering'), ('owner', 'Dorilton Capital')]


3

In [29]:
queryString = """
SELECT DISTINCT ?team ?owner
WHERE { 

?t wdt:P31 wd:Q10497835;
     wdt:P749 ?o .
?t sc:name ?team .
?o sc:name ?owner .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('team', 'McLaren'), ('owner', 'McLaren Technology Group')]
[('team', 'Aston Martin F1 Team'), ('owner', 'Aston Martin Lagonda')]
[('team', 'Scuderia Ferrari'), ('owner', 'Ferrari')]
[('team', 'Enzo Coloni Racing Car Systems'), ('owner', 'Scuderia Coloni')]


4

In [30]:
queryString = """
SELECT DISTINCT ?team ?owner
WHERE { 

?t wdt:P31 wd:Q10497835;
     wdt:P355 ?o .
?t sc:name ?team .
?o sc:name ?owner .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('team', 'Red Bull Racing'), ('owner', 'Red Bull Junior Team')]
[('team', 'Scuderia Ferrari'), ('owner', 'Ferrari Driver Academy')]


2

Probably a useful query is to take the union between `P749` and `P127` since the two relations produced disjoint output. Not care about `P355`

In [31]:
queryString = """
SELECT DISTINCT ?team ?owner
WHERE { 

?t wdt:P31 wd:Q10497835;
     wdt:P749|wdt:P127 ?o .
?t sc:name ?team .
?o sc:name ?owner .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('team', 'McLaren'), ('owner', 'McLaren Technology Group')]
[('team', 'Aston Martin F1 Team'), ('owner', 'Aston Martin Lagonda')]
[('team', 'Scuderia Ferrari'), ('owner', 'Ferrari')]
[('team', 'Red Bull Racing'), ('owner', 'Red Bull GmbH')]
[('team', 'Alpine F1 Team'), ('owner', 'Renault')]
[('team', 'Enzo Coloni Racing Car Systems'), ('owner', 'Scuderia Coloni')]
[('team', 'Williams Grand Prix Engineering'), ('owner', 'Dorilton Capital')]


7

## Task 3
How many F1 drivers and F1 teams exist in each country?

Looking for the relations of the team

In [32]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 

?t wdt:P31 wd:Q10497835;
    ?pr ?o .
?pr sc:name ?prop .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P9279'), ('prop', 'Egapro gender equality index')]
[('pr', 'http://www.wikidata.org/prop/direct/P1056'), ('prop', 'product or material produced')]
[('pr', 'http://www.wikidata.org/prop/direct/P112'), ('prop', 'founded by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1128'), ('prop', 'employees')]
[('pr', 'http://www.wikidata.org/prop/direct/P118'), ('prop', 'league')]
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('prop', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P127'), ('prop', 'owned by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('prop', 'participant in')]
[('pr', 'http://www.wikidata.org/prop/direct/P1365'), ('prop', 'replaces')]
[('pr', 'http://www.wikidata.org/prop/direct/P1366'), ('prop', 'replaced by')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('prop', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1382'), ('prop', 'partially coincident with')]
[('pr', 'http:

73

Found `P17` that is the country

In [33]:
queryString = """
SELECT DISTINCT ?country count(DISTINCT ?t) AS ?count
WHERE { 

?t wdt:P31 wd:Q10497835;
    wdt:P17 ?o .
?o sc:name ?country .
}
GROUP BY ?o ?country
ORDER BY DESC (?count)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'United Kingdom'), ('count', '26')]
[('country', 'Italy'), ('count', '16')]
[('country', 'France'), ('count', '6')]
[('country', 'Russia'), ('count', '2')]
[('country', 'Germany'), ('count', '2')]
[('country', 'Netherlands'), ('count', '2')]
[('country', 'Japan'), ('count', '2')]
[('country', 'Spain'), ('count', '1')]
[('country', 'Switzerland'), ('count', '1')]
[('country', 'Australia'), ('count', '1')]
[('country', 'Malaysia'), ('count', '1')]


11

Same for the driver

In [34]:
queryString = """
SELECT DISTINCT ?pr ?prop
WHERE { 

?t wdt:P106 wd:Q10841764;
    ?pr ?o .
?pr sc:name ?prop .
FILTER (REGEX (?prop,\".*coun.*\") || REGEX (?prop,\".*nat.*\") )
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P103'), ('prop', 'native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P109'), ('prop', 'signature')]
[('pr', 'http://www.wikidata.org/prop/direct/P1559'), ('prop', 'name in native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P268'), ('prop', 'Bibliothèque nationale de France ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1532'), ('prop', 'country for sport')]
[('pr', 'http://www.wikidata.org/prop/direct/P27'), ('prop', 'country of citizenship')]
[('pr', 'http://www.wikidata.org/prop/direct/P4453'), ('prop', 'Argentine Senate member ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('prop', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P6234'), ('prop', 'Biographie nationale de Belgique ID')]


9

Use `P27`, country of citizenship

In [35]:
queryString = """
SELECT DISTINCT ?country count(DISTINCT ?t) AS ?count
WHERE { 

?t wdt:P106 wd:Q10841764;
    wdt:P27 ?o .
?o sc:name ?country .
}
GROUP BY ?o ?country
ORDER BY DESC (?count) 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('country', 'United States of America'), ('count', '245')]
[('country', 'United Kingdom'), ('count', '172')]
[('country', 'Germany'), ('count', '58')]
[('country', 'Belgium'), ('count', '28')]
[('country', 'South Africa'), ('count', '27')]
[('country', 'Argentina'), ('count', '25')]
[('country', 'Italy'), ('count', '19')]
[('country', 'Spain'), ('count', '18')]
[('country', 'Austria'), ('count', '18')]
[('country', 'Kingdom of the Netherlands'), ('count', '17')]
[('country', 'Canada'), ('count', '16')]
[('country', 'United Kingdom of Great Britain and Ireland'), ('count', '16')]
[('country', 'France'), ('count', '14')]
[('country', 'Finland'), ('count', '9')]
[('country', 'Kingdom of Italy'), ('count', '9')]
[('country', 'Ireland'), ('count', '7')]
[('country', 'Brazil'), ('count', '6')]
[('country', 'Portugal'), ('count', '5')]
[('country', 'Mexico'), ('count', '5')]
[('country', 'Australia'), ('count', '3')]
[('country', 'German Democratic Republic'), ('count', '3')]
[('coun

34

## Task 4

Compare number of F1 drivers to their nationality

4.1 Is there some driver that joined a team in their own country?

4.2 Which team employed the larger number of drivers?

4.3 For each country , which are the teams and drivers with the highest number of F1 race participations

<h3> Task 4.1 </h3> Is there some driver that joined a team in their own country? First ask if there is one, if yes run the select query to see the results.

In [36]:
queryString = """
ASK
WHERE { 

?t wdt:P31 wd:Q10497835;
    wdt:P17 ?o .
?d wdt:P106 wd:Q10841764;
    wdt:P54 ?t;
    wdt:P27 ?o .
?t sc:name ?team .
?d sc:name ?driver .
?o sc:name ?country .

}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

In [37]:
queryString = """
SELECT DISTINCT ?driver ?team ?country
WHERE { 

?t wdt:P31 wd:Q10497835;
    wdt:P17 ?o .
?d wdt:P106 wd:Q10841764;
    wdt:P54 ?t;
    wdt:P27 ?o .
?t sc:name ?team .
?d sc:name ?driver .
?o sc:name ?country .

}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('driver', 'David Coulthard'), ('team', 'Red Bull Racing'), ('country', 'United Kingdom')]
[('driver', 'Alexander Albon'), ('team', 'Red Bull Racing'), ('country', 'United Kingdom')]
[('driver', 'Jackie Stewart'), ('team', 'Tyrrell Racing'), ('country', 'United Kingdom')]
[('driver', 'Enrico Bertaggia'), ('team', 'Forti'), ('country', 'Italy')]
[('driver', 'Tony Brooks'), ('team', 'British Racing Motors'), ('country', 'United Kingdom')]
[('driver', 'Roy Salvadori'), ('team', 'British Racing Motors'), ('country', 'United Kingdom')]
[('driver', 'Richard Attwood'), ('team', 'British Racing Motors'), ('country', 'United Kingdom')]
[('driver', 'Jackie Stewart'), ('team', 'British Racing Motors'), ('country', 'United Kingdom')]
[('driver', 'John Surtees'), ('team', 'British Racing Motors'), ('country', 'United Kingdom')]
[('driver', 'Mike Spence'), ('team', 'British Racing Motors'), ('country', 'United Kingdom')]


10

<h3> Task 4.2 </h3> Which team employed the larger number of drivers?

Write the first query to count drivers per team.

In [38]:
queryString = """

SELECT ?team count(?d) AS ?count
WHERE { 

?t wdt:P31 wd:Q10497835.

?d wdt:P106 wd:Q10841764;
    wdt:P54 ?t.

?t sc:name ?team .
?d sc:name ?driver .

}
GROUP BY ?t ?team
ORDER BY DESC (?count)
"""

print("Results")
run_query(queryString)

Results
[('team', 'Scuderia Ferrari'), ('count', '15')]
[('team', 'British Racing Motors'), ('count', '14')]
[('team', 'McLaren'), ('count', '14')]
[('team', 'Williams Grand Prix Engineering'), ('count', '13')]
[('team', 'Alfa Romeo'), ('count', '10')]
[('team', 'Brabham'), ('count', '9')]
[('team', 'Tyrrell Racing'), ('count', '5')]
[('team', 'Scuderia Toro Rosso'), ('count', '5')]
[('team', 'Red Bull Racing'), ('count', '5')]
[('team', 'Mercedes'), ('count', '4')]
[('team', 'March Engineering'), ('count', '4')]
[('team', 'Renault F1 Team'), ('count', '3')]
[('team', 'Benetton Formula'), ('count', '3')]
[('team', 'Equipe Matra Sports'), ('count', '3')]
[('team', 'Sauber'), ('count', '2')]
[('team', 'Arrows'), ('count', '2')]
[('team', 'Minardi'), ('count', '2')]
[('team', 'Andrea Moda Formula'), ('count', '2')]
[('team', 'Haas F1 Team'), ('count', '2')]
[('team', 'Jordan Grand Prix'), ('count', '1')]
[('team', 'Zakspeed'), ('count', '1')]
[('team', 'Racing Point F1 Team'), ('count', '

34

Select the max

In [39]:
queryString = """
SELECT ?team ?max 
WHERE{
    {
        SELECT max(?count) AS ?max WHERE{
        SELECT  ?team count(?d) AS ?count
        WHERE { 

        ?t wdt:P31 wd:Q10497835.

        ?d wdt:P106 wd:Q10841764;
            wdt:P54 ?t.

        ?t sc:name ?team .
        ?d sc:name ?driver .

        }
        GROUP BY ?t ?team
        }
    }
    {
        SELECT ?team count(?d) AS ?count
        WHERE { 

        ?t wdt:P31 wd:Q10497835.

        ?d wdt:P106 wd:Q10841764;
            wdt:P54 ?t.

        ?t sc:name ?team .
        ?d sc:name ?driver .

        }
        GROUP BY ?t ?team
    }
    
    FILTER (?count = ?max)
}
"""

print("Results")
run_query(queryString)

Results
[('team', 'Scuderia Ferrari'), ('max', '15')]


1

<h3> Task 4.3 </h3> For each country , which are the teams and drivers with the highest number of F1 race participations

First look for some relations

In [40]:
#relation with sports season
queryString = """

SELECT DISTINCT ?p ?ista
WHERE { 
?a wdt:P3450 wd:Q1968.
 ?o  ?p ?a.

?p sc:name ?ista .

}

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('ista', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('ista', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('ista', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('ista', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('ista', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('ista', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('ista', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('ista', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('ista', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('ista', 'main subject')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('ista', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P2499'), ('ista', 'league level above')]
[('p', 'http://www.wikidata.org/p

13

Found:
- `P1346`: winner
- `P361`: part of
- `P1344` participant in

In [41]:
#relation with the winner of sports season
queryString = """

SELECT DISTINCT ?ist 
WHERE { 
?a wdt:P3450 wd:Q1968.
 ?o wdt:1346 ?a.
?o wdt:P31 ?ist.

}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
Empty


0

In [42]:
#relation with the part of sports season
queryString = """

SELECT DISTINCT ?ist ?ista
WHERE { 
?a wdt:P3450 wd:Q1968.
 ?o wdt:P361 ?a.
?o wdt:P31 ?ist.
?ist sc:name ?ista .

}
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('ist', 'http://www.wikidata.org/entity/Q7854'), ('ista', 'Argentine Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q7993'), ('ista', 'Korean Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q53869933'), ('ista', 'sport result')]
[('ist', 'http://www.wikidata.org/entity/Q9214'), ('ista', 'Detroit Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q7961'), ('ista', 'Brazilian Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q192909'), ('ista', 'scandal')]
[('ist', 'http://www.wikidata.org/entity/Q7994'), ('ista', 'European Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q7997'), ('ista', 'French Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q7998'), ('ista', 'German Grand Prix')]
[('ist', 'http://www.wikidata.org/entity/Q8019'), ('ista', 'British Grand Prix')]


10

Found that `P361` relate a grand prix with a sport season

In [43]:
queryString = """

SELECT DISTINCT ?sup ?ista
WHERE { 
?a wdt:P3450 wd:Q1968.
 ?o wdt:P361 ?a.
?o wdt:P31 ?ist.
?ist wdt:P279+ ?sup.
?sup sc:name ?ista .

FILTER(REGEX(?ista,\".*[F,f]ormula [O,o]ne.*\"))
}
LIMIT 100

"""

print("Results")
run_query(queryString)

Results
[('sup', 'http://www.wikidata.org/entity/Q11924610'), ('ista', 'Formula One Grand Prix')]
[('sup', 'http://www.wikidata.org/entity/Q65742449'), ('ista', 'Formula One race')]


2

Look in details the F1 race and the F1 Grand Prix

In [44]:
#F1 race
queryString = """

SELECT DISTINCT ?a ?ista
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q65742449.

?a sc:name ?ista .
}
LIMIT 100

"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q16868870'), ('ista', '1951 San Remo Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q16868871'), ('ista', '1937 San Remo Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q16868873'), ('ista', '1947 San Remo Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q3113944'), ('ista', '1937 Pescara Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q97139307'), ('ista', '2020 Tuscan Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q97651788'), ('ista', '2020 Eifel Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q424206'), ('ista', '1957 Pescara Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q3113947'), ('ista', '1938 Pescara Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q640852'), ('ista', '1932 Pescara Grand Prix')]


9

In [45]:
#F1 Grand Prix
queryString = """

SELECT DISTINCT ?a ?ista
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610.

?a sc:name ?ista .
}
LIMIT 100

"""

print("Results")
run_query(queryString)

Results
[('a', 'http://www.wikidata.org/entity/Q17195719'), ('ista', '2014 Singapore Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q3113863'), ('ista', '1935 French Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q23045163'), ('ista', '2016 Japanese Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q593732'), ('ista', '1994 Spanish Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q595512'), ('ista', '1963 United States Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q595598'), ('ista', '1965 United States Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q596312'), ('ista', '1987 Detroit Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q664771'), ('ista', '2006 Turkish Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q664818'), ('ista', '2007 Turkish Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q1764779'), ('ista', '1951 Dutch Grand Prix')]
[('a', 'http://www.wikidata.org/entity/Q2767091'), ('ista', '1949 Australian Grand Prix')]
[('a', 'http:

100

Wonderful! `?a wdt:P31/wdt:P279+ wd:Q11924610.` this gives me all the effective gran prix.
Look for some relations

In [46]:
queryString = """

SELECT DISTINCT ?pr ?ista
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610;
    ?pr ?o.
?pr sc:name ?ista .

}

"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P8284'), ('ista', 'motorsportstats.com event ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('ista', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('ista', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1448'), ('ista', 'official name')]
[('pr', 'http://www.wikidata.org/prop/direct/P15'), ('ista', 'route map')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('ista', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1705'), ('ista', 'native label')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('ista', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P181'), ('ista', 'taxon range map image')]
[('pr', 'http://www.wikidata.org/prop/direct/P2283'), ('ista', 'uses')]
[('pr', 'http://www.wikidata.org/prop/direct/P242'), ('ista', 'locator map image')]
[('pr', 'http://www.wikidata.org/prop/direct/P2581'), ('ista', 'BabelNet ID')]
[('p

47

Found `P710`: participant. Maybe this is a driver participated in a Grand Prix

In [47]:
queryString = """

SELECT DISTINCT ?o ?driver count(*) as ?count
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610;
    wdt:P710 ?d.
?d wdt:P106 wd:Q10841764.
?d sc:name ?driver .

}
GROUP BY ?o ?driver
ORDER BY DESC (?count)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('driver', 'George Russell'), ('count', '7')]
[('driver', 'Charles Leclerc'), ('count', '7')]
[('driver', 'Sebastian Vettel'), ('count', '7')]
[('driver', 'Valtteri Bottas'), ('count', '7')]
[('driver', 'Lewis Hamilton'), ('count', '7')]
[('driver', 'Lando Norris'), ('count', '7')]
[('driver', 'Antonio Giovinazzi'), ('count', '7')]
[('driver', 'Max Verstappen'), ('count', '7')]
[('driver', 'Pierre Gasly'), ('count', '7')]
[('driver', 'Carlos Sainz Jr'), ('count', '7')]
[('driver', 'Lance Stroll'), ('count', '7')]
[('driver', 'Kimi Räikkönen'), ('count', '7')]
[('driver', 'Fernando Alonso'), ('count', '5')]
[('driver', 'Alexander Albon'), ('count', '2')]
[('driver', 'Romain Grosjean'), ('count', '1')]
[('driver', 'Alberto Ascari'), ('count', '1')]


16

Frustrating this graph. Very few data available.

In [48]:
queryString = """

SELECT DISTINCT count(DISTINCT ?a) as ?count
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610.
FILTER EXISTS{?a wdt:P710 ?d.}

}
"""

print("Results")
run_query(queryString)

Results
[('count', '8')]


1

In [49]:
queryString = """

SELECT DISTINCT count(*) as ?count
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610.
FILTER NOT EXISTS{?a   wdt:P710 ?d.}

}
"""

print("Results")
run_query(queryString)

Results
[('count', '1321')]


1

In fact the participant available in the grand prix is only 8 over 1329 grand prix in the history..

Look on the other direction

In [50]:
queryString = """

SELECT DISTINCT ?driver count(?gp) AS ?count
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610. 

?d wdt:P106 wd:Q10841764;
    wdt:P1344 ?a.
?a sc:name ?gp .
?d sc:name ?driver .

}
GROUP BY ?d ?driver
ORDER BY DESC (?count)


"""

print("Results")
run_query(queryString)

Results
[('driver', 'Romain Grosjean'), ('count', '20')]
[('driver', 'Fernando Alonso'), ('count', '6')]
[('driver', 'Niki Lauda'), ('count', '2')]
[('driver', 'Lee Wallard'), ('count', '2')]


4

Looking for the team

In [51]:
queryString = """

SELECT DISTINCT ?t ?team count(?a) as ?count
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610;
    wdt:P710 ?t.

?t wdt:P31 wd:Q10497835.
?t sc:name ?team .

}
GROUP BY ?t ?team
ORDER BY DESC (?count)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [52]:
queryString = """

SELECT DISTINCT ?team count(?gp) AS ?count
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610. 

?t wdt:P31 wd:Q10497835;
    wdt:P1344 ?a.
?a sc:name ?gp .
?t sc:name ?team .

}
GROUP BY ?t ?team
ORDER BY DESC (?count)


"""

print("Results")
run_query(queryString)

Results
Empty


0

Looking for team property associated to a grand prix

In [53]:
queryString = """

SELECT DISTINCT ?pr ?prop
WHERE { 
?a wdt:P31/wdt:P279+ wd:Q11924610. 

?t wdt:P31 wd:Q10497835.
?a   ?pr ?t.
?pr sc:name ?prop .

}
LIMIT 100


"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('prop', 'winner')]


1

I have only the information about the winner, so no info about participations.